# Overview
This notebook builds off of the previous notebook where we played with our LLM-As-A-Judge prompt to align it to our own grading preferences. In this notebook, instead of trying to align our scores like 4(a), we're trying to maximize our scores!

#### What Metrics Should I Care About.
This is a bit simpler than previous notebooks. We're essentially trying to get all 5s from our grading rubric. The higher the average score, the better we like our prompt. The validation dataset already contains our relevant context and we aren't focused on information retrieval.


# What Will We Do?
* Curate a dataset of questions and relevant context (we've created one already)
* Reuse our grading rubric from the previous notebook
* Invoke our model using our validation dataset to generate answers
* Generate scores to understand how well our prompt works.

At this end of this notebook, we should have a prompt that works very well for the types of answers we're looking for.

# Load Validation Dataset
This dataset contains the query and relevant context. 

In [ ]:
import pandas as pd

eval_df = pd.read_csv('../data/eval-datasets/4(b)_prompt_validation.csv')

In [ ]:
eval_df

# Evaluation Helper Classes
Because we're reusing a lot of the same code for calling bedrock within the RAG portion and the evaluation portion, it makes sense to push that to a base class and use inheritance to reuse code. The following functions allow us to perform the RAG and then perform the validation in a 2 step process


In [ ]:
import boto3
import pandas as pd
import re
from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Dict, Any

class BaseBedrockClient:
    def __init__(self, user_prompt: str, system_prompt: str, model_id: str, hyper_params: dict):
        self.client = boto3.client('bedrock-runtime')
        self.user_prompt = user_prompt
        self.system_prompt = system_prompt
        self.model_id = model_id
        self.hyper_params = hyper_params

    def create_chat_payload(self, inputs: dict) -> list[dict]:
        prompt = self.user_prompt.format(**inputs)
        return [{"role": "user", "content": [{"text": prompt}]}]

    def call(self, messages: list[dict]) -> str:
        response = self.client.converse(
            modelId=self.model_id,
            messages=messages,
            inferenceConfig=self.hyper_params,
            system=[{"text": self.system_prompt}]
        )
        return response['output']['message']['content'][0]['text']

    def call_threaded(self, message_lists: List[List[Dict[str, Any]]]) -> List[str]:
        future_to_position = {}
        with ThreadPoolExecutor(max_workers=5) as executor:
            for i, request in enumerate(message_lists):
                future = executor.submit(self.call, request)
                future_to_position[future] = i
            
            responses = [None] * len(message_lists)
            for future in as_completed(future_to_position):
                position = future_to_position[future]
                try:
                    response: str = future.result()
                    responses[position] = response
                except Exception as exc:
                    print(f"Request at position {position} generated an exception: {exc}")
                    responses[position] = None
        return responses

class RAGClient(BaseBedrockClient):
    def __init__(self, user_prompt: str, system_prompt: str, model_id: str, hyper_params: dict):
        super().__init__(user_prompt, system_prompt, model_id, hyper_params)

    def extract_response(self, llm_output: str) -> str:
        response_match = re.search(r'<response>(.*?)</response>', llm_output, re.DOTALL)
        return response_match.group(1).strip() if response_match else "No response found"

    def process(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        message_lists = [self.create_chat_payload({
            "query_text": row["query_text"],
            "context": row["context"]
        }) for _, row in df.iterrows()]
        
        responses = self.call_threaded(message_lists)
        df['llm_response'] = [self.extract_response(r) for r in responses]
        return df

class EvaluationClient(BaseBedrockClient):
    def __init__(self, user_prompt: str, system_prompt: str, model_id: str, hyper_params: dict):
        super().__init__(user_prompt, system_prompt, model_id, hyper_params)

    def extract_score_and_thinking(self, llm_output: str) -> tuple:
        thinking_match = re.search(r'<thinking>(.*?)</thinking>', llm_output, re.DOTALL)
        score_match = re.search(r'<score>(.*?)</score>', llm_output, re.DOTALL)

        thinking = thinking_match.group(1).strip() if thinking_match else "No thinking found"
        score = float(score_match.group(1)) if score_match else None
        
        return score, thinking

    def evaluate(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        message_lists = [self.create_chat_payload({
            "query_text": row["query_text"],
            "context": row["context"],
            "llm_response": row["llm_response"]
        }) for _, row in df.iterrows()]
        
        responses = self.call_threaded(message_lists)

        llm_scores = []
        llm_thinking = []

        for response in responses:
            if response is not None:
                score, thinking = self.extract_score_and_thinking(response)
                llm_scores.append(score)
                llm_thinking.append(thinking)
            else:
                llm_scores.append(None)
                llm_thinking.append("Error occurred during processing")

        df['grade'] = llm_scores
        df['reasoning'] = llm_thinking
        
        return df

# Define RAG Prompt
Before we evaluate anything, we need to construct a prompt that can take in context and generate answers. This example below is purposefully not good. It's here to set a baseline for which we can improve on. 

In [ ]:
# System Prompt
RAG_SYSTEM_PROMPT = """You are an advanced AI assistant specialized in Retrieval Augmented Generation (RAG).
Your primary function is to provide accurate, concise, and relevant answers based solely on the given context.
Follow these guidelines strictly:

1. Use only information from the provided context. Do not introduce external knowledge or make assumptions.
2. Ensure your answers are complete, addressing all aspects of the question using available information.
3. Be extremely concise. Use as few words as possible while maintaining clarity and completeness.
4. Maintain 100% accuracy based on the given context. If the context doesn't contain enough information to answer fully, state this clearly.
5. Structure your responses for maximum clarity. Use bullet points or numbered lists when appropriate.
6. If the context contains technical information, explain it in simple terms as if speaking to a non-technical person.
7. Do not apologize or use phrases like "Based on the context provided" or "According to the information given".
8. If asked about something not in the context, simply state "The provided context does not contain information about [topic]."

Your goal is to achieve the highest possible score on context utilization, completeness, conciseness, accuracy, and clarity."""

# User Prompt
RAG_USER_PROMPT = """Answer the following question using only the provided context:

<query>
{query_text}
</query>

<context>
{context}
</context>

Instructions:
1. Read the question and context carefully.
2. Formulate a concise and accurate answer based solely on the given context.
3. Ensure your response is clear and easily understandable to a non-technical person.
4. Do not include any information not present in the context.
5. If the context doesn't contain relevant information, state this clearly and concisely.
6. Place your response in <response></response> tags."""

# Reuse Rubric
We will reuse the rubric from our previous notebook that gave us the most consistant scores.

In [ ]:
# System Prompt
RUBRIC_SYSTEM_PROMPT = """You are an expert judge evaluating Retrieval Augmented Generation (RAG) applications.
Your task is to evaluate given answers based on context and questions using the criteria provided.
Evaluation Criteria (Score either 0 or 1 for each, total score is the sum):
1. Context Utilization: Does the answer use only information provided in the context, without introducing external or fabricated details?
2. Completeness: Does the answer thoroughly address all key elements of the question based on the available context, without significant omissions?
3. Conciseness: Does the answer efficiently use words to address the question and avoid unnecessary redundancy?
4. Accuracy: Is the answer factually correct based on the given context?
5. Clarity: Is the answer easy to understand and follow?
Your role is to provide a fair and thorough evaluation for each criterion, explaining your reasoning clearly."""

# User Prompt
RUBRIC_USER_PROMPT = """Please evaluate the following RAG response:

Question:
<query_text>
{query_text}
</query_text>

Generated answer:
<llm_response>
{llm_response}
</llm_response>

Context:
<context>
{context}
</context>

Evaluation Steps:
1. Carefully read the provided context, question, and answer.
2. For each evaluation criterion, assign a score of either 0 or 1:
   - Context Utilization
   - Completeness
   - Conciseness
   - Accuracy
   - Clarity
3. Provide a clear explanation for each score, referencing specific aspects of the response.
4. Calculate the total score by adding up the points awarded (minimum 0, maximum 5).
5. Present your evaluation inside <thinking></thinking> tags.
6. Include individual criterion scores (0 or 1) in the thinking tags and the total score inside <score></score> tags.
7. Ensure your response is valid XML and provides a comprehensive evaluation.

Example Output Format:
<thinking>
Context Utilization: 1 - The answer strictly uses information from the context without introducing external details.
Completeness: 1 - The response covers all key elements of the question based on the available context.
Conciseness: 1 - The answer is helpful and doesn't repeat the same information more than once.
Accuracy: 0 - The model introduced facts that were not present in the context.
Clarity: 1 - The response is clear and easy to follow.
</thinking>
<score>4</score>

Please provide your detailed evaluation."""

In [ ]:
# Define different models we can use to evaluate. 
SONNET_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
HAIKU_ID = "anthropic.claude-3-haiku-20240307-v1:0"

# Initialize RAG Client
rag_client = RAGClient(
    RAG_USER_PROMPT, 
    RAG_SYSTEM_PROMPT, 
    HAIKU_ID, 
    {"temperature": 0.5, "maxTokens": 4096}
)

# Initialize Eval Client
eval_client = EvaluationClient(
    RUBRIC_USER_PROMPT, 
    RUBRIC_SYSTEM_PROMPT, 
    HAIKU_ID, 
    {"temperature": 0.7, "maxTokens": 4096}
)

In [ ]:
# Generate RAG responses
rag_df = rag_client.process(eval_df)

# Evaluate RAG responses
experiment_1_df = eval_client.evaluate(rag_df)

In [ ]:
import pandas as pd
import numpy as np
from textwrap import fill

class PromptEvaluator:
    def __init__(self, df):
        self.df = df
        self.grades = df['grade'].astype(float)
    
    def calculate_metrics(self):
        return {
            'Mean': np.mean(self.grades),
            'Median': np.median(self.grades),
            'Standard Deviation': np.std(self.grades),
            'Minimum Grade': np.min(self.grades),
            'Maximum Grade': np.max(self.grades)
        }
    
    def generate_report(self):
        metrics = self.calculate_metrics()
        report = "Prompt Evaluation Report\n"
        report += "========================\n\n"
        
        for metric, value in metrics.items():
            report += f"{metric}: {value:.2f}\n"
        
        return report
    
    def analyze_grade_distribution(self):
        return self.df['grade'].value_counts().sort_index()

    def pretty_print_lowest_results(self, n=3, width=80):
        lowest_results = self.df.nsmallest(n, 'grade')
        for index, row in lowest_results.iterrows():
            print(f"{'='*width}\n")
            print(f"Grade: {row['grade']}\n")
            print("Query Text:")
            print(fill(row['query_text'], width=width))
            print("\nLLM Response:")
            print(fill(row['llm_response'], width=width))
            print("\nReasoning:")
            print(fill(row['reasoning'], width=width))
            print(f"\n{'='*width}\n")

In [ ]:
# Assuming your dataframe is named 'df'
evaluator = PromptEvaluator(experiment_1_df)

# Generate and print the report
print(evaluator.generate_report())

# Analyze grade distribution
print(evaluator.analyze_grade_distribution())

# Experiment 1 Results

Here was my results. Your results may vary. Remember, LLMs are non-deterministic. Your outputs might vary.
```bash
Mean: 5.00
Median: 5.00
Standard Deviation: 0.00
Minimum Grade: 5.00
Maximum Grade: 5.00
```

Based on my results, the model is performing extremely well. This is a relatively simple task and we can guarantee accurate context in this task, it makes sense we'd get 100% or close to it. Because we're getting the accuracy we care about, it doesn't make sense to iterate much further for now

# Conclusion
In this notebook, we did a bit of prompt engineering with our RAG prompt to get the performance we were looking for in our models response. It's worth noting that a basic RAG example doesn't require much prompt tuning. However, when we go to evaluate the E2E system, we can feel confident that any issues that arise are not because of our prompt!

# Next Steps
Move to the final E2E Notebook to evaluate our entire system.
